# Using Domain and Task Adaptive Pretraining with Tranformer Adapters
--Talk about the basis of the project--

## Installation
--We need to create a enviorment.yaml to install all the correct libraries--

In [ ]:
# test saving

In [ ]:
# Test cell number 1

In [1]:
# hi this is tony can you see it
!pip install numpy
!pip install pandas
# !pip install transformers --You don't want to have both transformers and adapter-transformers, it caueses issues. fyi adapter-transformers is actually a fork of transformers!
!pip install -U adapter-transformers
!pip install datasets

     |████████████████████████████████| 3.9 MB 8.7 MB/s 
     |████████████████████████████████| 596 kB 57.5 MB/s 
     |████████████████████████████████| 77 kB 6.8 MB/s 
     |████████████████████████████████| 6.6 MB 55.0 MB/s 
     |████████████████████████████████| 895 kB 54.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 27.7 MB/s 
     |████████████████████████████████| 136 kB 56.5 MB/s 
     |████████████████████████████████| 1.1 MB 60.0 MB/s 
     |████████████████████████████████| 212 kB 57.3 MB/s 
     |████████████████████████████████| 127 kB 63.3 MB/s 
     |████████████████████████████████| 271 kB 67.8 MB/s 
     |████████████████████████████████| 144 kB 71.1 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urlli

In [38]:

from datasets import list_datasets, load_dataset, list_metrics, load_metric
from transformers import RobertaTokenizer
from transformers import (RobertaConfig, RobertaModelWithHeads, RobertaModel, 
  RobertaAdapterModel, TrainingArguments, AdapterTrainer, EvalPrediction)
from transformers import DataCollatorForLanguageModeling
import numpy as np
import torch

# Load and Process Data

In [16]:
# cs_domain_name = 's2orc'
# cs_domain_dataset = load_dataset(cs_domain_name)
# print(cs_domain_dataset.num_rows)

scierc_name = 'nsusemiehl/SciERC'
scierc_dataset = load_dataset(scierc_name)
# print(scierc_dataset.num_rows)

# acl_arc_name = 'zapsdcn/citation_intent'
# acl_arc_dataset = load_dataset(acl_arc_name)
# print(acl_arc_dataset.num_rows)
# print(scierc_dataset)

Using custom data configuration nsusemiehl--SciERC-b7b410772d85970a
Reusing dataset json (/root/.cache/huggingface/datasets/json/nsusemiehl--SciERC-b7b410772d85970a/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/3 [00:00<?, ?it/s]

In [17]:
# scierc_dataset.max

In [29]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

def encode_batch(batch):
  """Encodes a batch of input data using the model tokenizer."""
  return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=128)

def add_masked_labels(examples):
  examples["labels"] = examples["input_ids"].copy()
  return examples

def create_masked_dataset(dataset):
  # Encode the input data
  # NOTE: num_proc does not seem to work, for some reason it can't find the tokenizer
  dataset_masked = dataset.map(
      encode_batch, 
      batched=True, 
      remove_columns=dataset['train'].column_names)
  
  dataset_masked = dataset_masked.map(add_masked_labels, batched=True)
  dataset_masked.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
  return dataset_masked

def create_finetune_dataset(dataset):
  # Encode the input data
  dataset_finetune = dataset.map(encode_batch, batched=True)
  # The transformers model expects the target class column to be named "labels"
  dataset_finetune = dataset_finetune.rename_column("label", 'labels')
  # Transform to pytorch tensors and only output the required columns
  label_to_int = { "COMPARE": 0, "CONJUNCTION": 1, "EVALUATE-FOR": 2, "FEATURE-OF": 3, "HYPONYM-OF": 4, "PART-OF": 5, "USED-FOR": 6,}
  dataset_finetune = dataset_finetune.map(lambda example: {"labels": label_to_int[example['labels']]})
  dataset_finetune.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
  return dataset_finetune

scierc_dataset_masked = create_masked_dataset(scierc_dataset)
scierc_dataset_finetune = create_finetune_dataset(scierc_dataset)

# Collater adds padding in the form of EOS tokens, makes data augmentations of random masking ('mlm_probability)
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

loading file https://huggingface.co/roberta-base/resolve/main/vocab.json from cache at /root/.cache/huggingface/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-base/resolve/main/merges.txt from cache at /root/.cache/huggingface/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/roberta-base/resolve/main/tokenizer_config.json from cache at None
loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd

  0%|          | 0/3219 [00:00<?, ?ex/s]

  0%|          | 0/974 [00:00<?, ?ex/s]

  0%|          | 0/455 [00:00<?, ?ex/s]

# Initialize Model

In [19]:
config = RobertaConfig.from_pretrained(
    "roberta-base",
    # num_labels=num_of_labels,
)
model = RobertaAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)
# Add adapter
model.add_adapter("sci_erc")
# Add a matching language model head
model.add_masked_lm_head(
    "sci_erc",
)
# Activate the adapter
# model.set_active_adapters("sci_erc")
model.train_adapter('sci_erc')

training_args = TrainingArguments(
    learning_rate=2e-5,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_steps=10,
    output_dir="./training_output/pretraining",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=True,
    evaluation_strategy = 'steps',
    # load_best_model_at_end = True,
)

# Adding Support for Tensorboard, supposedly you don't have to do this, but I find that it doesn't work
# from torch.utils.tensorboard import SummaryWriter
# from transformers.integrations import TensorBoardCallback
# writer = SummaryWriter()
# writer = TensorBoardCallback(writer)

trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=scierc_dataset_masked["train"],
    eval_dataset=scierc_dataset_masked["validation"],
    data_collator=data_collator
     
)

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https://huggingface.co/roberta-base/resolve/main/pytorch_model.bin from 

In [20]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3219
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 202


***** Running Evaluation *****
  Num examples = 455
  Batch size = 16
***** Running Evaluation *****
  Num examples = 455
  Batch size = 16
***** Running Evaluation *****
  Num examples = 455
  Batch size = 16
***** Running Evaluation *****
  Num examples = 455
  Batch size = 16
***** Running Evaluation *****
  Num examples = 455
  Batch size = 16
***** Running Evaluation *****
  Num examples = 455
  Batch size = 16
***** Running Evaluation *****
  Num examples = 455
  Batch size = 16
***** Running Evaluation *****
  Num examples = 455
  Batch size = 16
***** Running Evaluation *****
  Num examples = 455
  Batch size = 16
***** Running Evaluation *****
  Num examples = 455
  Batch size = 16
***** Running Evaluation *****
  Num examples = 455
  Batch size = 16
***** Running Evaluation *****
  Num examples = 455
  Batch size = 16
***** Running Evaluation *****
  Num examples = 455
  Batch size = 16
***** Running Evaluation *****
  Num examples = 455
  Batch size = 16
***** Running Evalua

TrainOutput(global_step=202, training_loss=12.40911869011303, metrics={'train_runtime': 159.7065, 'train_samples_per_second': 20.156, 'train_steps_per_second': 1.265, 'total_flos': 215534383370496.0, 'train_loss': 12.40911869011303, 'epoch': 1.0})

In [21]:
model.save_pretrained("tony_pretrained")

Configuration saved in tony_pretrained/config.json
Model weights saved in tony_pretrained/pytorch_model.bin


# Fine Tune

In [40]:
# config = RobertaConfig.from_pretrained(
#     "tony_pretrained",
#     num_labels=7,
# )
# model = RobertaAdapterModel.from_pretrained(
#     "tony_pretrained",
#     config=config,
# )

model.add_classification_head(
    "sci_erc",
    num_labels=7,
    overwrite_ok=True,
    id2label={0:'COMPARE', 1:'CONJUNCTION', 2:'EVALUATE-FOR', 
              3:'FEATURE-OF', 4:'HYPONYM-OF', 5:'PART-OF', 6:'USED-FOR'}
)

Adding head 'sci_erc' with config {'head_type': 'classification', 'num_labels': 7, 'layers': 2, 'activation_function': 'tanh', 'label2id': {'COMPARE': 0, 'CONJUNCTION': 1, 'EVALUATE-FOR': 2, 'FEATURE-OF': 3, 'HYPONYM-OF': 4, 'PART-OF': 5, 'USED-FOR': 6}, 'use_pooler': False, 'bias': True}.


In [42]:
training_args = TrainingArguments(
    learning_rate=1e-4,
    num_train_epochs=50,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    output_dir="./training_output",
    overwrite_output_dir=True,
    # The next line is important to ensure the dataset labels are properly passed to the model
    remove_unused_columns=False,
    evaluation_strategy = 'steps',
    # load_best_model_at_end = True,
)

# adding f1 metric
metric = load_metric('f1')
def compute_accuracy(EvalPrediction):
  logits, labels = EvalPrediction
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels, average= 'macro')
# def compute_accuracy(p: EvalPrediction):
#   preds = np.argmax(p.predictions, axis=1)
#   return {"acc": (preds == p.label_ids).mean()}
trainer = AdapterTrainer(
    model=model,
    args=training_args,
    train_dataset=scierc_dataset_finetune["train"],
    eval_dataset=scierc_dataset_finetune["validation"],
    compute_metrics=compute_accuracy,    
)

using `logging_steps` to initialize `eval_steps` to 200
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [43]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3219
  Num Epochs = 50
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 5050


***** Running Evaluation *****
  Num examples = 455
  Batch size = 32
